In [1]:
!pip install tensorflow-datasets

    100% |████████████████████████████████| 51kB 2.0MB/s 


In [0]:
import math
import io
import os
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import tree
from tqdm import tqdm
import operator
import pandas as pd
#import keras
#import h5py
#import cPickle

In [3]:
# RUN ONCE
# Run Runtime->Reset ALl Runtimes to reset
!git clone https://github.com/tensorflow/models
os.chdir('models/tutorials/image/cifar10/')

Cloning into 'models'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 23133 (delta 21), reused 24 (delta 10), pack-reused 23094
Receiving objects: 100% (23133/23133), 562.90 MiB | 29.31 MiB/s, done.
Resolving deltas: 100% (13510/13510), done.
Checking out files: 100% (2883/2883), done.


In [0]:
import cifar10
import cifar10_input
# Fix for flags from Burak's Textual Invariants colab.
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [5]:
cifar10.maybe_download_and_extract()

>> Downloading cifar-10-binary.tar.gz 100.0%
Successfully downloaded cifar-10-binary.tar.gz 170052171 bytes.


In [6]:
# Get test and train images.
with tf.Graph().as_default():
  train_images_bin, train_labels_bin = cifar10_input.inputs(False, "/tmp/cifar10_data/cifar-10-batches-bin", 50000)
  test_images_bin, test_labels_bin = cifar10_input.inputs(True, "/tmp/cifar10_data/cifar-10-batches-bin", 10000)
  with tf.train.MonitoredSession() as sess:
    test_images, test_labels = sess.run([test_images_bin, test_labels_bin])
    train_images, train_labels = sess.run([train_images_bin, train_labels_bin])

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Q

In [0]:
BATCH_SIZE = cifar10.FLAGS.batch_size

def create_model(t_images, shape=None):
  # If t_images is None then it creates a placeholder for feeding
  # images. The placeholder has the provided shape.
  #
  # Creates model with images supplied by the t_images tensor.
  if t_images is not None:  
    t_images_pl = tf.placeholder_with_default(t_images, shape=t_images.shape)
  else:
    t_images_pl = tf.placeholder(shape)
  # Build a Graph that computes the logits predictions from the
  # inference model.
  t_logits = cifar10.inference(t_images_pl)
  return t_images_pl, t_logits

def get_prediction(sess, tensor, t_images_pl, images, batch_size=BATCH_SIZE):
  """
  Evaluate given tensor by feeding the provided images.
  TODO: Currently partial batches cannot be evaluated so len(images) must
  be a multiple of batch_size.
  """
  def get_prediction_batch(image_batch):
    feed = {t_images_pl: np.array(image_batch)}
    return sess.run(tensor, feed_dict=feed)
  
  def get_final_batch(images, batch_size):
    pad_value = np.zeros(images[0].shape)
    final_batch = images[int(len(images)/batch_size)*batch_size:]
    pad_size = batch_size-len(final_batch)
    pad = [pad_value for _ in range(pad_size)]
    final_batch = np.concatenate(tuple([final_batch, pad]), axis=0)
    return final_batch, pad_size
  
  n = len(images)
  image_batches = [images[i*batch_size:(i+1)*batch_size] for i in range(int(n/batch_size))]
  padding_size = 0
  if (n % batch_size) != 0:
    # Pad the remainting entries and create an additional batch
    final_batch, pad_size = get_final_batch(images, batch_size)
    image_batches += [final_batch]
  batch_predictions = [get_prediction_batch(b) for b in tqdm(image_batches)]
  # remove padding
  batch_predictions = batch_predictions[:-1] + [batch_predictions[-1][:-pad_size]]
  res = np.concatenate(tuple(batch_predictions), axis=0)
  assert res.shape[0] == images.shape[0]
  return res
  
def train_model(save_steps=500):
  """Train CIFAR-10 for a number of steps."""
  with tf.Graph().as_default():
    t_global_step = tf.train.get_or_create_global_step()

    # Get images and labels for CIFAR-10.
    # Force input pipeline to CPU:0 to avoid operations sometimes ending up on
    # GPU and resulting in a slow down.
    with tf.device('/cpu:0'):
      t_images, t_labels = cifar10.distorted_inputs()

    # Build a Graph that computes the logits predictions from the
    # inference model.
    t_images_pl, t_logits = create_model(t_images)

    # Calculate loss.
    t_loss = cifar10.loss(t_logits, t_labels)

    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    t_train_op = cifar10.train(t_loss, t_global_step)
    saver_hook = tf.train.CheckpointSaverHook(
      checkpoint_dir='./checkpoints',
      save_secs=None,
      save_steps=save_steps,
      saver=tf.train.Saver(),
      checkpoint_basename='cifar_model.ckpt',
      scaffold=None)
    with tf.train.MonitoredTrainingSession(hooks=[saver_hook]) as sess:
      while not sess.should_stop():
        l, step, _ = sess.run([t_loss, t_global_step, t_train_op])
        if step % save_steps == 0:
          test_logits = get_prediction(sess, t_logits, t_images_pl, test_images)
          test_accuracy = np.mean(np.equal(np.argmax(test_logits, axis=1), test_labels))
          print "Loss at step %d: %f" % (step , l)
          print "Test accuracy at step %d: %f" % (step, test_accuracy)

## Train a new CIFAR10 model

In [0]:
train_model()

In [0]:
ckpt_path_name = "./checkpoints/cifar_model.ckpt-84000"
from google.colab import files
files.download(ckpt_path_name + '.data-00000-of-00001')
files.download(ckpt_path_name + '.index')
files.download(ckpt_path_name + '.meta')

In [0]:
!ls ./checkpoints

checkpoint
cifar_model.ckpt-1.data-00000-of-00001
cifar_model.ckpt-1.index
cifar_model.ckpt-1.meta
cifar_model.ckpt-400.data-00000-of-00001
cifar_model.ckpt-400.index
cifar_model.ckpt-400.meta
cifar_model.ckpt-600.data-00000-of-00001
cifar_model.ckpt-600.index
cifar_model.ckpt-600.meta
cifar_model.ckpt-65100.data-00000-of-00001
cifar_model.ckpt-65100.index
cifar_model.ckpt-65100.meta
cifar_model.ckpt-65200.data-00000-of-00001
cifar_model.ckpt-65200.index
cifar_model.ckpt-65200.meta
cifar_model.ckpt-65300.data-00000-of-00001
cifar_model.ckpt-65300.index
cifar_model.ckpt-65300.meta
cifar_model.ckpt-65400.data-00000-of-00001
cifar_model.ckpt-65400.index
cifar_model.ckpt-65400.meta
cifar_model.ckpt-700.data-00000-of-00001
cifar_model.ckpt-700.index
cifar_model.ckpt-700.meta
cifar_model.ckpt-800.data-00000-of-00001
cifar_model.ckpt-800.index
cifar_model.ckpt-800.meta
cifar_model.ckpt-83500.data-00000-of-00001
cifar_model.ckpt-83500.index
cifar_model.ckpt-83500.meta
cifar_model.ckpt-84000.da

## Restore model from checkpoint

In [8]:
!mkdir -p ./restored_checkpoints
!wget https://github.com/hayesconverse/sym_convnn/raw/master/CIFAR10_colab_checkpoint/checkpoints_ataly_model/cifar_model.ckpt-84000.index -O ./restored_checkpoints/cifar_model.ckpt.index
!wget https://github.com/hayesconverse/sym_convnn/raw/master/CIFAR10_colab_checkpoint/checkpoints_ataly_model/cifar_model.ckpt-84000.meta -O ./restored_checkpoints/cifar_model.ckpt.meta
!wget https://github.com/hayesconverse/sym_convnn/raw/master/CIFAR10_colab_checkpoint/checkpoints_ataly_model/cifar_model.ckpt-84000.data-00000-of-00001 -O ./restored_checkpoints/cifar_model.ckpt.data-00000-of-00001

--2018-12-01 22:55:25--  https://github.com/hayesconverse/sym_convnn/raw/master/CIFAR10_colab_checkpoint/checkpoints_ataly_model/cifar_model.ckpt-84000.index
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/CIFAR10_colab_checkpoint/checkpoints_ataly_model/cifar_model.ckpt-84000.index [following]
--2018-12-01 22:55:25--  https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/CIFAR10_colab_checkpoint/checkpoints_ataly_model/cifar_model.ckpt-84000.index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1167 (1.1K) [application/octet-stream]
Sav

In [9]:
tf.reset_default_graph()

# From now on we work with the default graph.
t_images_pl, t_logits = create_model(test_images[:BATCH_SIZE])
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, './restored_checkpoints/cifar_model.ckpt')


INFO:tensorflow:Restoring parameters from ./restored_checkpoints/cifar_model.ckpt


In [14]:
train_predictions_all = get_prediction(sess, t_logits, t_images_pl, train_images)
train_predictions = np.argmax(train_predictions_all, axis=1)
test_predictions_all = get_prediction(sess, t_logits, t_images_pl, test_images)
test_predictions = np.argmax(test_predictions_all, axis=1)

# ACCURACY
print ""
print "Accuracy", np.mean(np.equal(test_predictions, test_labels))

100%|██████████| 79/79 [00:00<00:00, 91.22it/s]


Accuracy 0.8314


## Library for IG Attribution and Conductance

In [16]:
t_fc1 = sess.graph.get_tensor_by_name('local4/local4:0')
t_conv1 = sess.graph.get_tensor_by_name('conv1/conv1:0')
t_label = tf.placeholder(tf.int32)
t_neuron_id = tf.placeholder(tf.int32)
t_grad = tf.gradients(t_logits[:, t_label], t_images_pl)
print t_fc1.shape
print t_conv1.shape
#t_conv2 = sess.graph.get_tensor_by_name('import/h_conv2:0')
t_grad_neuron = tf.gradients(t_logits[:, t_label], t_fc1)[0]
t_grad_conductance = tf.gradients(t_fc1[:,t_neuron_id], t_images_pl, grad_ys=t_grad_neuron[:, t_neuron_id])
print t_fc1.graph
print t_logits.graph
print sess.graph

(128, 192)
(128, 24, 24, 64)


In [0]:
import copy

def attribute(inp, label, baseline=None, steps=50, use_top_label=False):
  def top_label(inp):
    return np.argmax(get_prediction(sess, t_logits, t_images_pl, [inp])[0])
  if baseline is None:
    baseline = 0*inp
  scaled_inputs = [baseline + (float(i)/steps)*(inp-baseline) for i in range(0, steps)]
  #feed_dict = {W_conv1: convWeightMatrix[0], b_conv1: convBiasMatrix[0], 
  #           W_conv2: convWeightMatrix[1], b_conv2: convBiasMatrix[1], 
  #           W_conv3: convWeightMatrix[2], b_conv3: convBiasMatrix[2], 
  #           W_conv4: convWeightMatrix[3], b_conv4: convBiasMatrix[3], 
  #           W_fc1: denseWeightMatrix[0], b_fc1: denseBiasMatrix[0], 
  #           W_fc2: denseWeightMatrix[1], b_fc2: denseBiasMatrix[1]}
  if use_top_label:
    feed_dict[x] = [inp]
    logits = sess.run(t_logits, feed_dict=feed_dict)[0]
    label = np.argmax(logits)
  feed_dict[x] = scaled_inputs
  feed_dict[t_label] = label
  grads, scores = sess.run([t_grad, t_logits], feed_dict=feed_dict)  # shapes: <steps+1>, <steps+1, inp.shape>
  integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
  print "FINAL SCORE", scores[-1][label]
  print "BASELINE SCORE", scores[0][label]
  print "SUM", np.sum(integrated_gradients), "DIFF", scores[-1][label] - scores[0][label]
  return integrated_gradients

def conductance(inp, label, neuron_id=None, baseline=None, steps=50):
  # neuron_id is the id of the neuron in layer t_fc1 through which conductance
  # must be computed. If None, vanilla IG is computed.
  if baseline is None:
    baseline = 0*inp
  scaled_inputs = [baseline + (float(i)/steps)*(inp-baseline) for i in range(0, steps)]
  #feed_dict = {W_conv1: convWeightMatrix[0], b_conv1: convBiasMatrix[0], 
  #           W_conv2: convWeightMatrix[1], b_conv2: convBiasMatrix[1], 
  #           W_conv3: convWeightMatrix[2], b_conv3: convBiasMatrix[2], 
  #           W_conv4: convWeightMatrix[3], b_conv4: convBiasMatrix[3], 
  #           W_fc1: denseWeightMatrix[0], b_fc1: denseBiasMatrix[0], 
  #           W_fc2: denseWeightMatrix[1], b_fc2: denseBiasMatrix[1]}
  feed_dict[x] = scaled_inputs
  feed_dict[t_label] = label
  if neuron_id != None:
    feed_dict[t_neuron_id] = neuron_id
    grads, scores = sess.run([t_grad_conductance, t_logits], feed_dict=feed_dict)  # shapes: <steps+1>, <steps+1, inp.shape>
    integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
    return integrated_gradients
  grads, scores = sess.run([t_grad, t_logits], feed_dict=feed_dict)  # shapes: <steps+1>, <steps+1, inp.shape>    
  integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
  print "FINAL SCORE", scores[-1][label]
  print "BASELINE SCORE", scores[0][label]
  print "SUM", np.sum(integrated_gradients), "DIFF", scores[-1][label] - scores[0][label]
  return integrated_gradients

## Library for Visualizing Images and Attributions

In [0]:
import PIL.Image
from IPython.display import clear_output, Image, display, HTML
import numpy as np
from cStringIO import StringIO

In [0]:
FONT_PATH='/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed.ttf'
IMAGE_SIZE=24

def mnist_to_rgb(mnist_img):
  """
  Transformsn an MNIST image (shape: <784>) to a grayscale
  RGB image (shape: <28,28,3>)
  """
  pixel_array = mnist_img.reshape(IMAGE_SIZE, IMAGE_SIZE)  # shape: 28,28
  rgb_image = np.transpose([pixel_array,pixel_array,pixel_array], axes=[1,2,0])
  return rgb_image

def cifar_to_rgb(cifar_img):
  rgb_image = cifar_img.reshape(IMAGE_SIZE, IMAGE_SIZE, 3) # shape: 32,32,3
  return rgb_image

def pil_img(a):
  '''Returns a PIL image created from the provided RGB array.
  '''
  a = np.uint8(a)
  return PIL.Image.fromarray(a)

def mnist_to_pil_img(inp):
  rgb_inp = 255*mnist_to_rgb(inp)
  vis_inp = pil_img(rgb_inp)
  return vis_inp  

def cifar_to_pil_img(inp):
  rgb_inp = 255*cifar_to_rgb(inp)
  vis_inp = pil_img(rgb_inp) 

def pil_fig(fig):
  # Returns a PIL image obtained from the provided PLT figure.
  buf = io.BytesIO()
  fig.savefig(buf, format='png')
  plt.close(fig)
  buf.seek(0)
  img = PIL.Image.open(buf)
  return img

def show_img(img, fmt='jpeg'):
  '''Displays the provided PIL image
  '''
  f = StringIO()
  img.save(f, fmt)
  display(Image(data=f.getvalue()))
 
def show_mnist_img(mnist_img):
  show_img(pil_img(255*mnist_to_rgb(mnist_img)))
  
def show_cifar_img(cifar_img):
  show_img(pil_img(255*cifar_to_rgb(cifar_img)))
  
def gray_scale(img):
  '''Converts the provided RGB image to gray scale.
  '''
  img = np.average(img, axis=2)
  return np.transpose([img, img, img], axes=[1,2,0])

def normalize(attrs, ptile=99):
  '''Normalize the provided attributions so that they fall between
     -1.0 and 1.0.
  '''
  h = np.percentile(attrs, ptile)
  l = np.percentile(attrs, 100-ptile)
  return np.clip(attrs/max(abs(h), abs(l)), -1.0, 1.0)    

def pil_text(strs, shape, start_h=10, start_w=10, font_size=18, color=(0, 0, 0)):
  # Returns a PIL image with the provided text.
  img = pil_img(255*np.ones(shape))
  draw = PIL.ImageDraw.Draw(img)
  font = PIL.ImageFont.truetype(FONT_PATH, font_size)
  h = start_h
  for s in strs: 
    draw.text((start_w,h), s, fill=color, font=font)
    h = h + 30
  return img

def combine(imgs, horizontal=True):
  # Combines the provided PIL Images horizontally or veritically
  if horizontal:
    w = np.sum([img.size[0]+10 for img in imgs])
    h = np.max([img.size[1] for img in imgs])
  else:
    w = np.max([img.size[0] for img in imgs])
    h = np.sum([img.size[1]+10 for img in imgs])
  final_img = PIL.Image.new('RGB', (w, h), color='white')
  pos = 0
  for img in imgs:
    if horizontal:
      final_img.paste(im=img, box=(pos,0))
      pos = pos+img.size[0]+10
    else:
      final_img.paste(im=img, box=(0,pos))
      pos = pos+img.size[1]+10
  return final_img

def visualize_attrs(img, attrs, ptile=99):
  '''Visaualizes the provided attributions by first aggregating them
    along the color channel to obtain per-pixel attributions and then
    scaling the intensities of the pixels in the original image in
    proportion to absolute value of these attributions.

    The provided image and attributions must of shape (224, 224, 3).
  '''
  if np.sum(attrs) == 0.0:
    # print "Attributions are all ZERO"
    return pil_img(0*img)
  attrs = gray_scale(attrs)
  attrs = abs(attrs)
  attrs = np.clip(attrs/np.percentile(attrs, ptile), 0,1)
  vis = img*attrs
  return pil_img(vis)
  
  
R=np.array([255,0,0])
G=np.array([0,255,0])
B=np.array([0,0,255])
def visualize_attrs2(img, attrs, pos_ch=G, neg_ch=R, ptile=99):
  '''Visaualizes the provided attributions by first aggregating them
     along the color channel and then overlaying the positive attributions
     along pos_ch, and negative attributions along neg_ch.

     The provided image and attributions must of shape (224, 224, 3).
  '''
  if np.sum(attrs) == 0.0:
    # print "Attributions are all ZERO"
    return pil_img(0*img)
  attrs = gray_scale(attrs)
  attrs = normalize(attrs, ptile)   
  pos_attrs = attrs * (attrs >= 0.0)
  neg_attrs = -1.0 * attrs * (attrs < 0.0)
  attrs_mask = pos_attrs*pos_ch + neg_attrs*neg_ch
  vis = 0.3*gray_scale(img) + 0.7*attrs_mask
  return pil_img(vis)

## Extracting Invariant Candidates

In [0]:
t_fc1 = sess.graph.get_tensor_by_name('local4/local4:0')
t_conv1 = sess.graph.get_tensor_by_name('conv1/conv1:0')

In [0]:
#Test getting suffixes and checking accuracy
t_fc1 = sess.graph.get_tensor_by_name('local4/local4:0')
t_conv1 = sess.graph.get_tensor_by_name('conv1/conv1:0')

T_LAYER = t_fc1
def fingerprint_suffix(images):
  print "Getting fingerprint for", T_LAYER.name
  return (get_prediction(sess, T_LAYER, t_images_pl, images)>0.0).astype('int')

In [18]:
train_suffixes = fingerprint_suffix(train_images)
test_suffixes = fingerprint_suffix(test_images)


  2%|▏         | 6/391 [00:00<00:06, 59.04it/s]

Getting fingerprint for local4/local4:0


 13%|█▎        | 10/79 [00:00<00:00, 93.95it/s]

Getting fingerprint for local4/local4:0


100%|██████████| 79/79 [00:00<00:00, 92.76it/s]


In [0]:
def describe_input(i, training=True):
  if training:
    print "Input", i
    print "Groundtruth", train_labels[i]
    print "Prediction", train_predictions[i]
    print "Fine-grained prediction", 10*train_labels[i] + train_predictions[i]
    show_cifar_img(train_images[i])
  else:
    print "Input", i
    print "Groundtruth", test_labels[i]
    print "Prediction", test_predictions[i]
    print "Fine-grained prediction", 10*test_labels[i] + test_predictions[i]
    show_cifar_img(test_images[i])

### Build the Decision Tree

In [20]:
# Basic decision tree
basic_estimator = tree.DecisionTreeClassifier()
basic_estimator.fit(train_suffixes, train_predictions)

# Evaluate basic_estimator on test data
basic_estimator_acc = (basic_estimator.predict(test_suffixes) == test_labels)
basic_estimator_agreement = (basic_estimator.predict(test_suffixes) == test_predictions)
print "Estimator accuracy", 1.0*np.sum(basic_estimator_acc)/len(test_suffixes)
print "Estimator agreement", 1.0*np.sum(basic_estimator_agreement)/len(test_suffixes)

Estimator accuracy 0.7838
Estimator agreement 0.8621


In [21]:
# Fine-grained predictions decision tree
fine_grained_predictions = 10*train_labels + train_predictions
fine_grained_estimator = tree.DecisionTreeClassifier()
fine_grained_estimator.fit(train_suffixes, fine_grained_predictions)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [0]:
# Decision tree per label
def get_relative_predictions(label):
  print "Create relative predictions for label:%d" % label
  res = np.zeros(train_predictions.shape)
  for i in range(len(train_predictions)):
    pred = train_predictions[i]
    gt = cifar_train_labels[i]
    if gt == label and pred == gt:
      res[i] = 0
    elif gt == label and pred != gt:
      res[i] = 1
    else:
      res[i] = 2
  print "Num correct: %d" % np.sum(res == 0)
  print "Num misclassified: %d" % np.sum(res == 1)
  print "Num others: %d" % np.sum(res == 2)
  return res

def get_relative_estimator(label):
  predictions = get_relative_predictions(label)
  print "Creating decision tree for label:%d" % label
  estimator = tree.DecisionTreeClassifier()
  estimator.fit(train_suffixes, predictions)
  return estimator

In [0]:
# SLOW; run only if you want to build relative estimators.
relative_estimators = [None for _ in range(10)]
for i in range(10):
  relative_estimators[i] = get_relative_estimator(i)

### Examine clusters/invariants

In [0]:
def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(neuron_ids, neuron_sig):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into cifar_train_images).
  return np.where((train_suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

def is_consistent_cluster(cluster, predictions):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = predictions[cluster[0]]
  for i in cluster:
    if predictions[i] != pred:
      return False
  return True

def is_misclassified(i):
  return train_predictions[i] != train_labels[i]

def visualize_conductances(img, label, neuron_ids, only_on=False):
  # Visualize the conductances for the provided image.
  # Args:
  # - img: the provided cifar image
  # - label: prediction label w.r.t. conductance must be computed
  # - neuron_ids: list of neurons indices from the suffix tensor for which
  #    conductances must be computed.
  # - only_on: If True then conductance is computed only for those neurons
  #    that are on for the given image. 
  vis = [cifar_to_pil_img(img)]
  suffix = fingerprint_suffix([img])
  for i, id in enumerate(neuron_ids):
    if only_on and suffix[i] != 1:
      continue  
    igc = conductance(img, label, neuron_id=id)
    # igc = conductances[id]
    vis.append(visualize_attrs2(255*cifar_to_rgb(img), cifar_to_rgb(igc)))
  return combine(vis)

def get_invariant(estimator, ref_id):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into cifar_train_images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = train_images[ref_id]
  ref_suffix = train_suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  neuron_sig = ref_suffix[neuron_ids]
  cluster = get_suffix_cluster(neuron_ids, neuron_sig)
  return cluster, neuron_ids, neuron_sig

def get_all_invariants(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]
  
  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
    paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print "Obtained all paths"
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants

def describe_cluster(cluster, neuron_ids):
  neuron_sig = train_suffixes[cluster[0]][neuron_ids]
  print "Num neurons in invariant", len(neuron_ids)
  print "Neuron id and signature", zip(neuron_ids, neuron_sig)
  print "Cluster size: ", len(cluster)
  print "Num misclassified", len([i for i in cluster if is_misclassified(i)])

def describe_all_invariants(all_invariants):
  df = []
  for cl, invs in all_invariants.iteritems():
    # Note the number of invariants, and size of the largest invariant cluster
    df.append([cl, sum([inv[2] for inv in invs]), len(invs), len([inv for inv in invs if inv[2]>=10]), invs[0][2]])
  df = pd.DataFrame(df, columns=['Prediction Class', 'Num Instances', 'Num Invariants', 'Num Invariants with cluster size >= 10', 'Size of largest invariant cluster'])
  return df

In [0]:
# Examine cluster/invariants containing a given reference input
# ref_id is the index of the reference input
ref_id =  2
print "### Reference Image ###"
describe_input(ref_id)
print "### Cluster ###"
cluster, neuron_ids, neuron_sig = get_invariant(basic_estimator, ref_id)
describe_cluster(cluster, neuron_ids)

# Visualize  10 inputs in the cluster
for i in cluster[:25]:
  describe_input(i)
  # show_img(visualize_conductances(mnist.train.images[i], train_predictions[i], neuron_ids, only_on=False))

In [0]:
b_all_invariants = get_all_invariants(basic_estimator)
df = describe_all_invariants(b_all_invariants)
print "Total num invariants:", df['Num Invariants'].sum()
print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
print df.to_string(index=False)

In [0]:
# Get all fine_grained_estimator invariants
fge_all_invariants = get_all_invariants(fine_grained_estimator)
# Print invariant stats
df = describe_all_invariants(fge_all_invariants)
print "Total num invariants:", df['Num Invariants'].sum()
print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
print df.to_string(index=False)



### Analyzing clusters of misclassified inputs

In [0]:
# Examine the cluster for a misclasification (Groundtruth: 4, Prediction: 49)
invs = b_all_invariants[0]
neuron_ids, neuron_sig, _ = invs[0]
cluster = get_suffix_cluster(neuron_ids, neuron_sig)
describe_cluster(cluster, neuron_ids)

# Visualize  10 inputs in the cluster
for i in cluster[:10]:
  describe_input(i)
  # show_img(visualize_conductances(mnist.train.images[i], train_predictions[i], neuron_ids, only_on=False))

## Confidence modelling

In [23]:
def get_estimator_confidence(estimator, suffixes, estimator_predictions, estimator_conf_labels, min_samples):
  estimator_leaf_nodes = estimator.apply(suffixes)
  # check that the leaf has valid label. (For instance, for a fine grained estimator
  # only labels 00, 11, .. are valid.)
  estimator_conf = [p in estimator_conf_labels for p in estimator_predictions]
  # check that the leaf has a minimum number of samples in its support
  estimator_conf *= estimator.tree_.n_node_samples[estimator_leaf_nodes] >= min_samples
  # check that the leaf is pure
  estimator_pure = np.array([ len(np.where(v != 0)[0]) == 1 for v in estimator.tree_.value[estimator_leaf_nodes][:,0,:]])
  estimator_conf *= estimator_pure
  return estimator_conf

def get_confident_accuracy(estimator, suffixes, orig_model_predictions, gt_labels, estimator_conf_labels, min_samples=10):
  """
  Args:
    estimator: the estimator used for confidence modeling
    suffixes: the suffixes of the examples over which we evaluate. Typically this
      would be 'test_suffixes'.
    orig_model_predictions: predictions of the original model for these examples. 
      Typically this would be 'test_predictions'.
    gt_labels: groundtruth labels for these examples. Typically this would be
      mnist.test.labels.
    estimator_conf_labels: prediction labels for the estimator which must be used
      for confidence labels. If the estimator predicts a label outside this set then
      the prediction on the example is not considered "confident". For the basic_estimator
      all labels are fine for confidence modeling. For the fine_grained estimator
      we would want to only use the "pure" labels for confident modeling. For
      instance, 00, 11, ... , for the MNIST fine_grained_estimator.
    min_samples: minimum number of samples for an estimator leaf for it to be
      considered confident
  """
  estimator_predictions = estimator.predict(suffixes)

  # The following are all binary vectors of shape <len(suffixes)>  
  orig_correct = (orig_model_predictions == gt_labels)
  estimator_conf = get_estimator_confidence(estimator, suffixes, estimator_predictions, estimator_conf_labels, min_samples)

  conf_frac = 1.0*np.sum(estimator_conf)/len(gt_labels)
  overall_acc = 1.0*np.sum(orig_correct)/len(gt_labels)
  conf_acc = 1.0*np.sum(orig_correct*estimator_conf)/np.sum(estimator_conf)
 
  return conf_frac, overall_acc, conf_acc

# BASIC_ESTIMATOR
res = get_confident_accuracy(basic_estimator, test_suffixes, test_predictions, test_labels, np.array(range(10)), min_samples = 100)
conf_frac, overall_acc,  conf_acc = res
print "BASIC ESTIMATOR"
print "Confident fraction", conf_frac
print "Overall accuracy", overall_acc
print "Confident accuracy", conf_acc

# FINE_GRAINED_ESTIMATOR
res = get_confident_accuracy(fine_grained_estimator, test_suffixes, test_predictions, test_labels, 10*np.array(range(10))+np.array(range(10)) , min_samples = 250)
conf_frac, overall_acc,  conf_acc = res

print ""
print "FINE GRAINED ESTIMATOR"
print "Confident fraction", conf_frac
print "Overall accuracy", overall_acc
print "Confident accuracy", conf_acc

BASIC ESTIMATOR
Confident fraction 0.6292
Overall accuracy 0.8314
Confident accuracy 0.9361093452

FINE GRAINED ESTIMATOR
Confident fraction 0.383
Overall accuracy 0.8314
Confident accuracy 0.978328981723


##Download and run adversarial examples

In [24]:
!mkdir downloads
!curl -sL https://github.com/mzweilin/EvadeML-Zoo/releases/download/v0.1/downloads.tar.gz | tar xzv -C downloads
!mkdir results
!curl -sL https://github.com/mzweilin/EvadeML-Zoo/releases/download/v0.1/results_MNIST_100_317f6_carlini.tar.gz | tar xzv -C results
!curl -sL https://github.com/mzweilin/EvadeML-Zoo/releases/download/v0.1/results_CIFAR-10_100_de671_densenet.tar.gz | tar xzv -C results
!curl -sL https://github.com/mzweilin/EvadeML-Zoo/releases/download/v0.1/results_ImageNet_100_a2749_mobilenet.tar.gz | tar xzv -C results

MagNet/
MagNet/defensive_models/
MagNet/defensive_models/MNIST_II
MagNet/defensive_models/MNIST_I
MagNet/defensive_models/CIFAR
trained_models/
trained_models/MNIST_pgdtrained.keras_weights.h5
trained_models/MNIST_pgdbase.keras_weights.h5
trained_models/CIFAR-10_carlini.keras_weights.h5
trained_models/MNIST_cleverhans.keras_weights.h5
trained_models/MNIST_cleverhans_adv_trained.keras_weights.h5
trained_models/MNIST_carlini.keras_weights.h5
MNIST_100_317f6_carlini/
MNIST_100_317f6_carlini/adv_examples/
MNIST_100_317f6_carlini/adv_examples/MNIST_100_317f6_carlini_jsma?targeted=next.pickle
MNIST_100_317f6_carlini/adv_examples/MNIST_100_317f6_carlini_carlinili?targeted=next&batch_size=1&max_iterations=1000&confidence=10.pickle
MNIST_100_317f6_carlini/adv_examples/MNIST_100_317f6_carlini_bim?eps=0.3&eps_iter=0.06.pickle
MNIST_100_317f6_carlini/adv_examples/MNIST_100_317f6_carlini_adaptive_carlini_l2?targeted=next&tf_squeezers=median_filter_2_2,binary_filter_0.5&distance_measure=l1&detector_

In [0]:
import h5py
import cPickle

#Load adversarial examples
#Questions: where are the labels? and how is this all normalized, if that matters?
targetNext1Pickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_carlinili?targeted=next&confidence=5.pickle", 'rb'))
targetNext1Data = targetNext1Pickle[0]
targetll1Pickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_carlinili?targeted=ll&confidence=5.pickle", 'rb'))
targetll1Data = targetll1Pickle[0]
targetNext2Pickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_carlinil2?targeted=next&batch_size=100&max_iterations=1000&confidence=5.pickle", 'rb'))
targetNext2Data = targetNext2Pickle[0]
targetll2Pickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_carlinil2?targeted=ll&batch_size=100&max_iterations=1000&confidence=5.pickle", 'rb'))
targetll2Data = targetll2Pickle[0]
targetNext0Pickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_carlinil0?targeted=next&confidence=5.pickle", 'rb'))
targetNext0Data = targetNext0Pickle[0]
targetll0Pickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_carlinil0?targeted=ll&confidence=5.pickle", 'rb'))
targetll0Data = targetll0Pickle[0]
targetNextJsmaPickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_jsma?targeted=next.pickle", 'rb'))
targetNextJsmaData = targetNextJsmaPickle[0]
targetllJsmaPickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_jsma?targeted=ll.pickle", 'rb'))
targetllJsmaData = targetllJsmaPickle[0]
fgsmPickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_fgsm?eps=0.0156.pickle", 'rb'))
fgsmData = fgsmPickle[0]
bimPickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_bim?eps=0.008&eps_iter=0.0012.pickle", 'rb'))
bimData = bimPickle[0]
deepfoolPickle = cPickle.load(open("results/CIFAR-10_100_de671_densenet/adv_examples/CIFAR-10_100_de671_densenet_deepfool?overshoot=10.pickle", 'rb'))
deepfoolData = deepfoolPickle[0]

In [27]:
#Select desired data here
centeredData = tf.image.resize_image_with_crop_or_pad(bimData, 24, 24)
adversarial_images = sess.run(centeredData)

adversarial_predictions_all = get_prediction(sess, t_logits, t_images_pl, adversarial_images)
adversarial_predictions = np.argmax(adversarial_predictions_all, axis=1)
adversarial_suffixes = fingerprint_suffix(adversarial_images)
adversarial_suffixes = (adversarial_suffixes>0).astype('int')

fge_adversarial_predictions = fine_grained_estimator.predict(adversarial_suffixes)
be_adversarial_predictions = basic_estimator.predict(adversarial_suffixes)

#Adjust confidence threshold here
be_adversarial_conf = get_estimator_confidence(basic_estimator, adversarial_suffixes, be_adversarial_predictions, np.array(range(10)), min_samples=10)
print "Number of confident adversarial examples from basic estimator", np.sum(be_adversarial_conf)
fge_adversarial_conf = get_estimator_confidence(fine_grained_estimator, adversarial_suffixes, fge_adversarial_predictions, 10*np.array(range(10))+np.array(range(10)), min_samples=10)
print "Number of confident adversarial examples from fine grained estimator", np.sum(fge_adversarial_conf)


100%|██████████| 1/1 [00:00<00:00, 32.51it/s]

Getting fingerprint for local4/local4:0
Number of confident adversarial examples from basic estimator 25
Number of confident adversarial examples from fine grained estimator 5


### Visualizing the Decision Tree

In [0]:
!apt-get install graphviz
!pip install graphviz
import graphviz

In [0]:
dot_data = tree.export_graphviz(basic_estimator, out_file=None) 
graph = graphviz.Source(dot_data)  
graph 